In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM 
# No activation for CuDNNLSTM - uses tanh

In [ ]:
# test with mnist data
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

In [18]:
# define the model architecture
model = Sequential()
model.add(LSTM(128,input_shape=(x_train.shape[1:]),activation='relu',return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10,activation='softmax'))

In [19]:
# optimizer
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)

In [20]:
# compile
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

In [ ]:
# train
model.fit(x_train,y_train,epochs=1)

In [23]:
# save model
tf.keras.models.save_model(model,'model')
# load model
model = tf.keras.models.load_model('model')

In [31]:
# test model
y_pred = model.predict(x_test)
y_pred_c = y_pred.argmax(axis=-1)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_c)

0.91105

In [33]:
# test with trajectories
# data generated using generate_random_trajectories.py

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM
import pickle
import numpy as np

with open('data.pkl','rb') as f:
    x_train,x_val,x_test,y_train,y_val,y_test = pickle.load(f)
x_train,x_val,x_test,y_train,y_val,y_test = np.array(x_train),np.array(x_val),np.array(x_test),np.array(y_train),np.array(y_val),np.array(y_test)

In [34]:
model = Sequential()
model.add(LSTM(128,input_shape=(x_train.shape[1:]),activation='relu',return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(4,activation='softmax'))

In [36]:
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3,validation_data=(x_val,y_val))

Train on 60000 samples, validate on 20000 samples
Epoch 1/3
60000/60000 [==============================] - 254s 4ms/step - loss: 0.5802 - acc: 0.7642 - val_loss: 0.4228 - val_acc: 0.8593
Epoch 2/3
60000/60000 [==============================] - 293s 5ms/step - loss: 0.4355 - acc: 0.8540 - val_loss: 0.3831 - val_acc: 0.8799
Epoch 3/3
60000/60000 [==============================] - 272s 5ms/step - loss: 0.3866 - acc: 0.8751 - val_loss: 0.3392 - val_acc: 0.8961


In [37]:
tf.keras.models.save_model(model,'model2')
# Model 2 has rotating data.  tested below,
model = tf.keras.models.load_model('model2')
y_pred = model.predict(x_test)
y_pred_c = y_pred.argmax(axis=-1)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_c)

0.8936

In [ ]:
# Test on real trajectories

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np


In [ ]:
# load data
df=pd.read_csv('rand_traj_200.csv')

In [ ]:
# process csv file to get individual trajectories
trajectories = {}
id_mapping={}
c=0
for index, row in df.iterrows():
    if row['id'] not in id_mapping:
        id_mapping[row['id']] = c+1
        c=c+1
    if id_mapping[row['id']] not in trajectories:
        trajectories[id_mapping[row['id']]] = []    
    trajectories[id_mapping[row['id']]].append([row['local_time_1'],row['x'],row['y'],row['speed']])

In [ ]:
# functions to normalize data based on the parameters used for the randomly generated data
# update SCALE_DISTANCE and SCALE_SPEED if updated there

SCALE_DISTANCE = 1500
SCALE_SPEED = 70

def get_seconds(t):
    import datetime
    import time
    x=time.strptime(t[11:19],'%H:%M:%S')
    return datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()

def kmph_to_mph(speed):
    return speed/1.609

def norm_speed(speed):
    return speed/SCALE_SPEED

def norm_distance(distance):
    return distance/1

In [ ]:
# extract normalized trajectories in x_test
# only trajectories that have between 12 and 24 points are considered here
x_test=[]
x_time = []
x_count=[]
for key in trajectories:
    t = trajectories[key]
    t_ = []
    if len(t)>0:
        start_time = get_seconds(t[0][0])
        start_x = t[0][1]
        start_y = t[0][2]
        
    for i in range(len(t)):
        t_.append([norm_distance(t[i][1]-start_x),norm_distance(t[i][2]-start_y),norm_speed(kmph_to_mph(t[i][3]))])
    
    if len(t_)>=12 and len(t_)<=24:
        if len(t_)>12:
            t_=t_[0:12]
        x_test.append(t_)

In [ ]:
# test
y_pred = model.predict(np.array(x_test))
y_pred_c = y_pred.argmax(axis=-1)
# accuracy: 0.8732394366197183